In [1]:
# Importing Libraries
import selenium
import pandas as pd
import time
from bs4 import BeautifulSoup

# Importing selenium webdriver 
from selenium import webdriver

# Importing required Exceptions which needs to handled
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException

#Importing requests
import requests

# importing regex
import re

# 1. Write a python program which searches all the product under a particular product from www.amazon.in.
The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search
for guitars.

In [52]:
#connect to webdriver
driver=webdriver.Chrome("chromedriver.exe")
time.sleep(2)

C:\Users\Nirav\AppData\Local\Temp/ipykernel_7436/2300614395.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome("chromedriver.exe")


In [53]:
# to get url
driver.get("https://www.amazon.in/")

In [54]:
time.sleep(2)
# input the keywords that user wants to search
user_input = input('Enter the product you want to search : ')

Enter the product you want to search : guitars


In [55]:
# to Locate searc_bar by id
search_bar = driver.find_element_by_id("twotabsearchtextbox") 
# to clear the search_bar
search_bar.clear() 
# to send user input to search bar
search_bar.send_keys(user_input)
 # to Locate search_button by xpath
search_button = driver.find_element_by_xpath('//div[@class="nav-search-submit nav-sprite"]/span/input')      
search_button.click()  

C:\Users\Nirav\AppData\Local\Temp/ipykernel_7436/3945557456.py:2: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search_bar = driver.find_element_by_id("twotabsearchtextbox")
C:\Users\Nirav\AppData\Local\Temp/ipykernel_7436/3945557456.py:8: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  search_button = driver.find_element_by_xpath('//div[@class="nav-search-submit nav-sprite"]/span/input')


# 2. In the above question, now scrape the following details of each product listed in first 3 pages of your
search results and save it in a data frame and csv. In case if any product has less than 3 pages in search
results then scrape all the products available under that product name. Details to be scraped are: "Brand
Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and
“Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [56]:
#collecting all the Product URLS
urls = []
for i in range(0,3):
    Page_urls=driver.find_elements_by_xpath("//a[@class='a-link-normal s-no-outline']")#collecting urls of all the laptop
    for i in Page_urls:
        urls.append(i.get_attribute('href'))
        
    #next button 
    nxt_btn=driver.find_element_by_xpath("//a[@class='s-pagination-item s-pagination-next s-pagination-button s-pagination-separator']") 
    url=nxt_btn.get_attribute('href')
    driver.get(url)
    time.sleep(2)

C:\Users\Nirav\AppData\Local\Temp/ipykernel_7436/1096546905.py:4: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  Page_urls=driver.find_elements_by_xpath("//a[@class='a-link-normal s-no-outline']")#collecting urls of all the laptop
C:\Users\Nirav\AppData\Local\Temp/ipykernel_7436/1096546905.py:9: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  nxt_btn=driver.find_element_by_xpath("//a[@class='s-pagination-item s-pagination-next s-pagination-button s-pagination-separator']")


In [58]:
#Making empty lists and scraping the required spots
Product_name = []
Brand_name= []
Ratings = []
No_Ratings = []
Price = []
Return = []
Expected_Delivery = []
Availability = []
Other_Details = []

#Start with for loop
for i in urls:
    driver.get(i)
    time.sleep(3)
    
    
    #Scraping product name
    try:
        prod=driver.find_element_by_xpath("//span[@id='productTitle']")
        Product_name.append(prod.text)
    except NoSuchElementException as e:
        Product_name.append("-")
        
        
    #Scraping brand name
    try:
        brand=driver.find_element_by_xpath("//div[@id='bylineInfo_feature_div']/div/a")
        Brand_name.append(brand.text)
    except NoSuchElementException as e:
        Brand_name.append("-")
        
        
     #Scraping Ratings
    try:
        rat=driver.find_element_by_xpath("//span[@id='acrPopover']")
        Ratings.append(rat.get_attribute("title"))   
    except NoSuchElementException as e:
        Ratings.append("-")
        
        
    #Scraping No of Ratings
    try:
        no_rat=driver.find_element_by_xpath("//a[@id='acrCustomerReviewLink']/span")
        No_Ratings.append(no_rat.text)
    except NoSuchElementException as e:
        No_Ratings.append("-")
        
        
    #Scraping Price
    try:
        pri=driver.find_element_by_xpath('//td[@class="a-span12"]')  
        Price.append(pri.text)
    except NoSuchElementException as e:
        Price.append("-")
        
        
    #Scraping Return/Exchange
    try:
        ret=driver.find_element_by_xpath("//div[@data-name='RETURNS_POLICY']/span/div[2]/a")
        Return.append(ret.text)
    except NoSuchElementException as e:
        Return.append("-")
        
     
    #Scraping  Expected_Delivery
    try:
        delivary=driver.find_element_by_xpath("//div[@id='ddmDeliveryMessage']/b")
        Expected_Delivery.append(delivary.text)
    except NoSuchElementException as e:
        Expected_Delivery.append("-")
        
        
    #Scraping Availability
    try:
        avai=driver.find_element_by_xpath("//div[@id='availability']/span")
        Availability.append(avai.text)
    except NoSuchElementException as e:
        Availability.append("-")
        
        
    #Scraping Other_Details
    try:
        details=driver.find_element_by_xpath("//ul[@class='a-unordered-list a-vertical a-spacing-mini']")
        Other_Details.append(details.text)
    except NoSuchElementException as e:
        Other_Details.append("-")
        
        
# To make dataframe
Guitar=pd.DataFrame({})
Guitar['Name'] = Product_name
Guitar['Brand'] = Brand_name
Guitar['Rating'] = Ratings
Guitar['No of Ratings'] = No_Ratings
Guitar['Price'] = Price
Guitar['Return'] = Return
Guitar['Expected_Delivery'] = Expected_Delivery
Guitar['Availability'] = Availability
Guitar['Other_Details'] = Other_Details
Guitar['Urls'] = urls

C:\Users\Nirav\AppData\Local\Temp/ipykernel_7436/2002154631.py:20: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  prod=driver.find_element_by_xpath("//span[@id='productTitle']")
C:\Users\Nirav\AppData\Local\Temp/ipykernel_7436/2002154631.py:28: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  brand=driver.find_element_by_xpath("//div[@id='bylineInfo_feature_div']/div/a")
C:\Users\Nirav\AppData\Local\Temp/ipykernel_7436/2002154631.py:36: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  rat=driver.find_element_by_xpath("//span[@id='acrPopover']")
C:\Users\Nirav\AppData\Local\Temp/ipykernel_7436/2002154631.py:44: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  no_rat=driver.find_element_by_xpath("//a[@id='acrCustomerRe

In [60]:
#Print dataframe
Guitar

,Name,Brand,Rating,No of Ratings,Price,Return,Expected_Delivery,Availability,Other_Details,Urls
0,Kadence Frontier guitar with Online Guitar lea...,Visit the Kadence Store,4.0 out of 5 stars,"1,486 ratings","₹5,399.00",7 Days Replacement,"Monday, May 30",,"★ Design - The High Gloss finish, Simple head ...",https://www.amazon.in/gp/slredirect/picassoRed...
1,Kadence Frontier Jumbo Semi Acoustic Guitar Wi...,Visit the Kadence Store,4.0 out of 5 stars,"1,218 ratings","₹7,348.00",7 Days Replacement,-,In stock.,QUALITY STRINGS ---The surface is coated with ...,https://www.amazon.in/gp/slredirect/picassoRed...
2,Vault Traveller 34 Inch Acoustic Guitar With B...,Visit the VAULT Store,3.9 out of 5 stars,344 ratings,-,7 Days Replacement,-,In stock.,"Size: 34"" Steel stringed | Number of Strings: ...",https://www.amazon.in/gp/slredirect/picassoRed...
3,"Blueberry 38C, 38"" Acoustic Guitar Kit with Ba...",Visit the blueberry Store,3.8 out of 5 stars,14 ratings,-,7 Days Replacement,-,In stock.,"Material: Top: linden wood, Side & Back: - lin...",https://www.amazon.in/gp/slredirect/picassoRed...
4,"Juârez Acoustic Guitar, 38 Inch Cutaway, 038C ...",Visit the JUAREZ Store,4.0 out of 5 stars,"15,901 ratings",-,7 Days Replacement,-,In stock.,"Black Glossy Finish, Number of Frets: 18, Acou...",https://www.amazon.in/Juarez-Acoustic-Cutaway-...
...,...,...,...,...,...,...,...,...,...,...
164,"Blueberry B-D38, 38 Inch Acoustic Guitar Kit I...",Visit the blueberry Store,3.9 out of 5 stars,13 ratings,-,7 Days Replacement,-,In stock.,"Material: Top: linden wood, Side & Back: - lin...",https://www.amazon.in/Blueberry-B-D38-Acoustic...
165,Saga SF-600GC Grand Concert Acoustic Guitar,Brand: SAGA,-,-,-,7 Days Replacement,-,In stock.,-,https://www.amazon.in/SF-600GC-Grand-Concert-A...
166,"JUAREZ Arpeggio 41 Inch Acoustic Guitar Kit, S...",Visit the JUAREZ Store,4.0 out of 5 stars,5 ratings,-,7 Days Replacement,-,In stock.,"Juarez Arpeggio 41 Inch Acoustic Guitar Kit, S...",https://www.amazon.in/JUAREZ-Arpeggio-Acoustic...
167,"Walden, Acoustic Guitar, Grand Auditorium with...",Brand: Walden,-,-,-,7 Days Replacement,-,In stock.,This Tobacco Burst Grand Auditorium acoustic-e...,https://www.amazon.in/Walden-Acoustic-Auditori...


In [227]:
driver.close()

# Write a python program to access the search bar and search button on images.google.com and scrape 10
images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’.

In [228]:
driver=webdriver.Chrome("chromedriver.exe")
time.sleep(2)

C:\Users\Nirav\AppData\Local\Temp/ipykernel_7436/1659935237.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome("chromedriver.exe")


In [229]:
driver.get("https://images.google.com/")

In [230]:
# empty list creating to scrap the detail
fruit_image = []
car_image = []
machineLearning_image = []
guitar_image = []
cakes_image = []

In [233]:
# iterating 3 types of image
for i in ['fruits','cars','machine learning','Guitar','cakes']:
    # opening google img website
    url = "https://images.google.com/?gws_rd=ssl"
    driver.get(url)
    time.sleep(4)
    
    
    #Scraping the fruits detail
    if i == 'fruits':
        # Accessing search bar and search button
        search_bar = driver.find_element_by_xpath("//input[@class='gLFyf gsfi']").send_keys(i)
        search_btn=driver.find_element_by_xpath("//span[@class= 'z1asCe MZy1Rb']")
        driver.execute_script('arguments[0].click()', search_btn)
        time.sleep(2)
        
        # first 10 image of fruit scraping
        for i in range(0,3):
            for i in driver.find_elements_by_xpath("//img[@class='rg_i Q4LuWd']"):
                if len(fruit_image) < 10:
                    fruit_image.append(i.get_attribute("src"))
                    time.sleep(2)
                else:
                    break
            driver.execute_script("window.scrollTo(0, 1000)")#scroll down the website
            time.sleep(4)
        
        
        
    #Scraping the cars detail    
    elif i == 'cars':
        # Accessing search bar and search button
        search_bar = driver.find_element_by_xpath("//input[@class='gLFyf gsfi']").send_keys(i)
        time.sleep(2)
        search_btn=driver.find_element_by_xpath("//span[@class= 'z1asCe MZy1Rb']")
        driver.execute_script('arguments[0].click()', search_btn)
        time.sleep(2)
        
        # first 10 image of car scraping
        for i in range(0,3):
            for i in driver.find_elements_by_xpath("//img[@class='rg_i Q4LuWd']"):
                if len(car_image) < 10:
                    car_image.append(i.get_attribute("src"))
                    time.sleep(2)
                else:
                    break
            driver.execute_script("window.scrollTo(0, 1000)") #scroll down the website
            time.sleep(4)
        
    #Scraping the machine learning detail     
    elif i == 'machine learning':
        # Accessing search bar and search button
        search_bar = driver.find_element_by_xpath("//input[@class='gLFyf gsfi']").send_keys(i)
        time.sleep(2)
        search_btn=driver.find_element_by_xpath("//span[@class= 'z1asCe MZy1Rb']")
        driver.execute_script('arguments[0].click()', search_btn)        
        time.sleep(2)
        
        # first 10 image of machinelearning scraping        
        for i in range(0,3):
            for i in driver.find_elements_by_xpath("//img[@class='rg_i Q4LuWd']"):
                if len(machineLearning_image) < 10:
                    machineLearning_image.append(i.get_attribute("src"))
                    time.sleep(2)
                else:
                    break
            driver.execute_script("window.scrollTo(0, 1000)")#scroll down the website
            time.sleep(4) 

   #Scraping the guitar detail
    if i == 'Guitar':
        # Accessing search bar and search button
        search_bar = driver.find_element_by_xpath("//input[@class='gLFyf gsfi']").send_keys(i)
        time.sleep(2)
        search_btn=driver.find_element_by_xpath("//span[@class= 'z1asCe MZy1Rb']")
        driver.execute_script('arguments[0].click()', search_btn)
        time.sleep(2)
        
        # first 10 image of fruit scraping
        for i in range(0,3):
            for i in driver.find_elements_by_xpath("//img[@class='rg_i Q4LuWd']"):
                if len(guitar_image) < 10:
                    guitar_image.append(i.get_attribute("src"))
                    time.sleep(2)
                else:
                    break
            driver.execute_script("window.scrollTo(0, 1000)")#scroll down the website
            time.sleep(4)
        
   #Scraping the cakes detail
    if i == 'cakes':
        # Accessing search bar and search button
        search_bar = driver.find_element_by_xpath("//input[@class='gLFyf gsfi']").send_keys(i)
        time.sleep(2)
        search_btn=driver.find_element_by_xpath("//span[@class= 'z1asCe MZy1Rb']")
        driver.execute_script('arguments[0].click()', search_btn)
        time.sleep(2)
        
        # first 10 image of fruit scraping
        for i in range(0,3):
            for i in driver.find_elements_by_xpath("//img[@class='rg_i Q4LuWd']"):
                if len(cakes_image) < 10:
                    cakes_image.append(i.get_attribute("src"))
                    time.sleep(2)
                else:
                    break
            driver.execute_script("window.scrollTo(0, 1000)")#scroll down the website
            time.sleep(4)
                            
            
            
#creating dataframe            
google = pd.DataFrame()
google['Fruits'] = fruit_image  
google['Cars'] = car_image
google['Machine Learning'] = machineLearning_image
google['Guitar'] = guitar_image
google['Cakes'] = cakes_image

google

C:\Users\Nirav\AppData\Local\Temp/ipykernel_7436/3942528621.py:12: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  search_bar = driver.find_element_by_xpath("//input[@class='gLFyf gsfi']").send_keys(i)
C:\Users\Nirav\AppData\Local\Temp/ipykernel_7436/3942528621.py:13: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  search_btn=driver.find_element_by_xpath("//span[@class= 'z1asCe MZy1Rb']")
C:\Users\Nirav\AppData\Local\Temp/ipykernel_7436/3942528621.py:19: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  for i in driver.find_elements_by_xpath("//img[@class='rg_i Q4LuWd']"):
C:\Users\Nirav\AppData\Local\Temp/ipykernel_7436/3942528621.py:33: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  search_bar = driver.find_elem

,Fruits,Cars,Machine Learning,Guitar,Cakes
0,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
1,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
2,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
3,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
4,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
5,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
6,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
7,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
8,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
9,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."


In [235]:
driver.close()

# Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on
www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be
scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”,
“Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the
details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [67]:
driver=webdriver.Chrome("chromedriver.exe")
time.sleep(2)

C:\Users\Nirav\AppData\Local\Temp/ipykernel_7436/1659935237.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome("chromedriver.exe")


In [68]:
driver.get("https://www.flipkart.com/")
time.sleep(3)

In [69]:
# to close login pop up
driver.find_element_by_xpath("//button[@class='_2KpZ6l _2doB4z']").click()

C:\Users\Nirav\AppData\Local\Temp/ipykernel_7436/2460521896.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath("//button[@class='_2KpZ6l _2doB4z']").click()


In [70]:
# to enter the item name which user wants to search
item = input(" Enter the name of Smartphone that has to be searched : ")

#giving input key word to search bar
serch_bar = driver.find_element_by_xpath("//div[@class='_3OO5Xc']//input")
serch_bar.send_keys(item)

# to click search button
srch_btn = driver.find_element_by_xpath("/html/body/div[1]/div/div[1]/div[1]/div[2]/div[2]/form/div/button")
srch_btn.click()
time.sleep(5)

 Enter the name of Smartphone that has to be searched : Oneplus Nord


C:\Users\Nirav\AppData\Local\Temp/ipykernel_7436/591919671.py:5: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  serch_bar = driver.find_element_by_xpath("//div[@class='_3OO5Xc']//input")
C:\Users\Nirav\AppData\Local\Temp/ipykernel_7436/591919671.py:9: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  srch_btn = driver.find_element_by_xpath("/html/body/div[1]/div/div[1]/div[1]/div[2]/div[2]/form/div/button")


In [71]:
# To get  urls of phones on 1st page
page1_urls = []
urls = driver.find_elements_by_class_name('s1Q9rs')
for url in urls:
    page1_urls.append(url.get_attribute("href"))

C:\Users\Nirav\AppData\Local\Temp/ipykernel_7436/481061456.py:3: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  urls = driver.find_elements_by_class_name('s1Q9rs')


In [72]:
len(page1_urls)

40

In [73]:
# to make emoty list for all features to be scraped
Smartphones = {}
Smartphones["Brand"] = []
Smartphones["Phone name"] = []
Smartphones["Colour"] = []
Smartphones["RAM"] = []
Smartphones["Storage(ROM)"] = []
Smartphones["Primary Camera"] = []
Smartphones["Secondary Camera"] = []
Smartphones["Display Size"] = []
Smartphones["Display Resolution"] = []
Smartphones["Processor"] = []
Smartphones["Processor Cores"] = []
Smartphones["Battery Capacity"] = []
Smartphones["Price"] = []
Smartphones["URL"] = []

In [74]:
# Scraping data from each url of page 1
for url in page1_urls:
    driver.get(url)                                                        
    print("Scraping URL = ", url)
    Smartphones['URL'].append(url)                                                          
    time.sleep(2)
    
    
    #Clicking on read more button
    try:
        read_more = driver.find_element_by_xpath('//button[@class="_2KpZ6l _1FH0tX"]')     
        read_more.click()
    except NoSuchElementException:
        print("Exception occured while moving to next page")
    
    
    #Scraping brand name of phone data
    try:
        brand_tags = driver.find_element_by_xpath('//span[@class="B_NuCI"]')      
        Smartphones["Brand"].append(brand_tags.text.split()[0])
    except NoSuchElementException:
        Smartphones['Brand'].append('-')
    
    
    #Scraping phone name data
    try:
        name_tags = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][1]/table/tbody/tr[3]/td[2]/ul/li')     
        Smartphones['Phone name'].append(name_tags.text)
    except NoSuchElementException:
        Smartphones['Phone name'].append('-')
    
    
    #Scraping phone color data
    try:
        color_tags = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][1]/table/tbody/tr[4]/td[2]/ul/li')      
        Smartphones['Colour'].append(color_tags.text)
    except NoSuchElementException:
        Smartphones['Colour'].append('-')
     
    
    #Scraping RAM data
    try:
        ram_tags = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][4]/table[1]/tbody/tr[2]/td[2]/ul/li')                
        Smartphones['RAM'].append(ram_tags.text)
    except NoSuchElementException:
        Smartphones['RAM'].append('-')
    
    
    #Scraping ROM data
    try:
        rom_tags = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][4]/table[1]/tbody/tr[1]/td[2]/ul/li')        
        Smartphones['Storage(ROM)'].append(rom_tags.text)
    except NoSuchElementException:
        Smartphones['Storage(ROM)'].append('-')
        
        
    #Scraping Primary camera data
    try:                                                                                    
        pri_tags = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[2]/td[2]/ul/li')
        Smartphones['Primary Camera'].append(pri_tags.text)
    except NoSuchElementException:
        Smartphones['Primary Camera'].append('-')
        
        
    #Scraping secondary camera data
    try:                                                                                    
        sec_tags = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[6]/td[1]')
        if sec_tags != "Secondary Camera" : 
            if driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[5]/td[1]').text == "Secondary Camera":
                sec_cam = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[5]/td[2]/ul/li')
            else :
                raise NoSuchElementException
        else :
            sec_cam = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[6]/td[2]/ul/li')
        Smartphones['Secondary Camera'].append(sec_cam.text)
    except NoSuchElementException:
        Smartphones['Secondary Camera'].append('-')
        
        
    #Scraping Display size data 
    try:
        disp_tags = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][2]/div')
        if disp_tags.text != "Display Features" : raise NoSuchElementException
        disp_size = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][2]/table[1]/tbody/tr[1]/td[2]/ul/li')  
        Smartphones['Display Size'].append(disp_size.text)
    except NoSuchElementException:
        Smartphones['Display Size'].append('-')
    
    
    #Scraping display resolution data
    try:
        dires_tags = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][2]/div')
        if dires_tags.text != "Display Features" : raise NoSuchElementException
        disp_reso = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][2]/table[1]/tbody/tr[2]/td[2]/ul/li')    
        Smartphones['Display Resolution'].append(disp_reso.text)
    except NoSuchElementException:
        Smartphones['Display Resolution'].append('-')
    
    
    
    #Scraping Processor data
    try:
        pro_tags = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[2]/td[1]')
        if pro_tags.text != "Processor Type" : raise NoSuchElementException
        processor = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[2]/td[2]/ul/li')   
        Smartphones['Processor'].append(processor.text)
    except NoSuchElementException:
        Smartphones['Processor'].append('-')
        
    
    
    #Scraping Processor core data    
    try:                                                                                     
        core_tags = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[3]/td[1]')
        if core_tags.text != "Processor Core" :
            core_tags = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[2]/td[1]')
            if core_tags.text != "Processor Core" : 
                raise NoSuchElementException
            else :
                cores = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[2]/td[2]/ul/li')
        else :
            cores = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][3]/table[1]/tbody/tr[3]/td[2]/ul/li')
        Smartphones['Processor Cores'].append(cores.text)
    except NoSuchElementException:
        Smartphones['Processor Cores'].append('-')
    
    
    
    #Scraping battery capacity data
    try:
        if driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/div').text != "Battery & Power Features" :
            if driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/div').text == "Battery & Power Features" :
                bat_tags = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/table/tbody/tr/td[1]')
                if bat_tags.text != "Battery Capacity" : raise NoSuchElementException
                bat_capa = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/table/tbody/tr/td[2]/ul/li')                
            elif driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/div').text == "Battery & Power Features" :
                bat_tags = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/table/tbody/tr/td[1]')
                if bat_tags.text != "Battery Capacity" : raise NoSuchElementException
                bat_capa = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/table/tbody/tr/td[2]/ul/li')
            else:
                raise NoSuchElementException
        else :
            bat_tags = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/table/tbody/tr/td[1]')
            if bat_tags.text != "Battery Capacity" : raise NoSuchElementException
            bat_capa = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/table/tbody/tr/td[2]/ul/li')              
        Smartphones['Battery Capacity'].append(bat_capa.text)
    except NoSuchElementException:
        Smartphones['Battery Capacity'].append('-')
        
        
        
    #Scraping Price data
    try:
        price_tags = driver.find_element_by_xpath('//div[@class="_30jeq3 _16Jk6d"]')      
        Smartphones['Price'].append(price_tags.text)
    except NoSuchElementException:
        Smartphones['Price'].append('-')
        

Scraping URL =  https://www.flipkart.com/oneplus-nord-ce-2-lite-5g-black-dusk-128-gb/p/itm7acae55b999e6?pid=MOBGDWF8GJTFAXVH&lid=LSTMOBGDWF8GJTFAXVH1ZEISR&marketplace=FLIPKART&q=Oneplus+Nord&store=tyy&srno=s_1_1&otracker=search&otracker1=search&fm=organic&iid=0ea05a4c-edd3-4dae-9874-73c7fb01473d.MOBGDWF8GJTFAXVH.SEARCH&ppt=hp&ppn=homepage&ssid=vzzma7414w0000001653589678831&qH=53863d4d978d5074


C:\Users\Nirav\AppData\Local\Temp/ipykernel_7436/3835993118.py:11: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  read_more = driver.find_element_by_xpath('//button[@class="_2KpZ6l _1FH0tX"]')
C:\Users\Nirav\AppData\Local\Temp/ipykernel_7436/3835993118.py:19: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  brand_tags = driver.find_element_by_xpath('//span[@class="B_NuCI"]')
C:\Users\Nirav\AppData\Local\Temp/ipykernel_7436/3835993118.py:27: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  name_tags = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][1]/table/tbody/tr[3]/td[2]/ul/li')
C:\Users\Nirav\AppData\Local\Temp/ipykernel_7436/3835993118.py:35: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  color_tags = dr

Scraping URL =  https://www.flipkart.com/oneplus-nord-2-5g-pac-man-256-gb/p/itmc88e4a702db46?pid=MOBGEYH6EJGAWMFT&lid=LSTMOBGEYH6EJGAWMFTY8ECT6&marketplace=FLIPKART&q=Oneplus+Nord&store=tyy&srno=s_1_2&otracker=search&otracker1=search&fm=organic&iid=0ea05a4c-edd3-4dae-9874-73c7fb01473d.MOBGEYH6EJGAWMFT.SEARCH&ppt=hp&ppn=homepage&ssid=vzzma7414w0000001653589678831&qH=53863d4d978d5074
Scraping URL =  https://www.flipkart.com/oneplus-nord-2-5g-gray-sierra-128-gb/p/itmc88e4a702db46?pid=MOBGDBFFA5EGECHQ&lid=LSTMOBGDBFFA5EGECHQNBGIQE&marketplace=FLIPKART&q=Oneplus+Nord&store=tyy&srno=s_1_3&otracker=search&otracker1=search&fm=organic&iid=0ea05a4c-edd3-4dae-9874-73c7fb01473d.MOBGDBFFA5EGECHQ.SEARCH&ppt=hp&ppn=homepage&ssid=vzzma7414w0000001653589678831&qH=53863d4d978d5074
Scraping URL =  https://www.flipkart.com/lowcost-asm-front-back-tempered-glass-oneplus-nord-ce-2-5g-camera-glass-screen-protector-5g/p/itm666d5ef3e042c?pid=ACCGCRAZ4MCFAZF6&lid=LSTACCGCRAZ4MCFAZF6PXQDQ2&marketplace=FLIPKART&q=

Scraping URL =  https://www.flipkart.com/mobile-mart-back-cover-oneplus-nord-ce-2-5g/p/itm912d9d287f99b?pid=ACCGCJFYRSPTQCNU&lid=LSTACCGCJFYRSPTQCNUWKPPQO&marketplace=FLIPKART&q=Oneplus+Nord&store=tyy&srno=s_1_22&otracker=search&otracker1=search&fm=organic&iid=0ea05a4c-edd3-4dae-9874-73c7fb01473d.ACCGCJFYRSPTQCNU.SEARCH&ppt=hp&ppn=homepage&ssid=vzzma7414w0000001653589678831&qH=53863d4d978d5074
Scraping URL =  https://www.flipkart.com/spazy-case-back-cover-oneplus-nord-ce-2-5g/p/itm7284c26adccdf?pid=ACCGE88AHRZGE8FH&lid=LSTACCGE88AHRZGE8FHAE30SE&marketplace=FLIPKART&q=Oneplus+Nord&store=tyy&srno=s_1_23&otracker=search&otracker1=search&fm=organic&iid=0ea05a4c-edd3-4dae-9874-73c7fb01473d.ACCGE88AHRZGE8FH.SEARCH&ppt=hp&ppn=homepage&ssid=vzzma7414w0000001653589678831&qH=53863d4d978d5074
Scraping URL =  https://www.flipkart.com/unirock-tempered-glass-guard-oneplus-nord-ce/p/itm9c0249fa211a5?pid=ACCGEKZBHCTYZPRV&lid=LSTACCGEKZBHCTYZPRVYVGO2P&marketplace=FLIPKART&q=Oneplus+Nord&store=tyy&srno=

In [75]:
#Checking lengths of all scraped data
print(len(Smartphones["Brand"]), len(Smartphones["Phone name"]), len(Smartphones["Colour"]), len(Smartphones["RAM"]), len(Smartphones["Storage(ROM)"]))

40 40 40 40 40


In [76]:
#Framing the data
df = pd.DataFrame.from_dict(Smartphones)
df

,Brand,Phone name,Colour,RAM,Storage(ROM),Primary Camera,Secondary Camera,Display Size,Display Resolution,Processor,Processor Cores,Battery Capacity,Price,URL
0,OnePlus,Nord CE 2 Lite 5G,Black Dusk,6 GB,128 GB,Primary Camera,-,16.74 cm (6.59 inch),2412 x 1080$$pixel,-,Octa Core,-,"₹19,830",https://www.flipkart.com/oneplus-nord-ce-2-lit...
1,OnePlus,Nord 2 5G,PAC-MAN,12 GB,256 GB,Primary Camera,-,16.33 cm (6.43 inch),2400 x 1080$$pixel,-,Octa Core,-,"₹37,890",https://www.flipkart.com/oneplus-nord-2-5g-pac...
2,OnePlus,Nord 2 5G,Gray Sierra,8 GB,128 GB,Primary Camera,-,16.33 cm (6.43 inch),1080 x 2400$$pixels,-,Dual Core,-,"₹28,892",https://www.flipkart.com/oneplus-nord-2-5g-gra...
3,LOWCOST,Front & Back,3,-,-,-,-,-,-,-,-,-,₹449,https://www.flipkart.com/lowcost-asm-front-bac...
4,OnePlus,Nord 2 5G,Blue Haze,8 GB,128 GB,Primary Camera,-,16.33 cm (6.43 inch),1080 x 2400$$pixels,-,Dual Core,-,"₹29,338",https://www.flipkart.com/oneplus-nord-2-5g-blu...
5,OnePlus,Nord CE 2 5G,Gray Mirror,8 GB,128 GB,Primary Camera,-,16.33 cm (6.43 inch),2400 x 1080$$pixel,-,Dual Core,-,"₹24,999",https://www.flipkart.com/oneplus-nord-ce-2-5g-...
6,Tommcase,"Oneplus Nord CE 2 5g, Oneplus Nord CE 2 5g Dee...",Metallic Black,-,-,-,-,-,-,-,-,-,₹549,https://www.flipkart.com/tommcase-back-cover-o...
7,Discoverz,OnePlus Nord CE 5G,black,-,-,-,-,-,-,-,-,-,₹499,https://www.flipkart.com/discoverz-back-cover-...
8,Micvir,OnePlus Nord 2 5G,Black,-,-,-,-,-,-,-,-,-,₹179,https://www.flipkart.com/micvir-back-cover-one...
9,NewSelect,OnePlus Nord,Transparent,-,-,-,-,-,-,-,-,-,₹179,https://www.flipkart.com/newselect-back-cover-...


In [78]:
driver.close()

# Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google
maps.

In [79]:
driver=webdriver.Chrome("chromedriver.exe")
time.sleep(2)

C:\Users\Nirav\AppData\Local\Temp/ipykernel_7436/1659935237.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome("chromedriver.exe")


In [80]:
driver.get("https://www.google.co.in/maps")
time.sleep(2)

In [81]:
#Sending keyword for seach bar and search button
city = input('Enter City name that has to be searched : ')
search_bar = driver.find_element_by_id("searchboxinput")                       
search_bar.clear()                                                             
time.sleep(2)
search_bar.send_keys(city)                                                     
search_btn = driver.find_element_by_id("searchbox-searchbutton")              
search_btn.click()                                                             
time.sleep(3)

try:
    url_string = driver.current_url
    print("URL Extracted: ", url_string)
    lat_lng = re.findall(r'@(.*)data',url_string)
    if len(lat_lng):
        lat_lng_list = lat_lng[0].split(",")
        if len(lat_lng_list)>=2:
            lat = lat_lng_list[0]
            lng = lat_lng_list[1]
        print("Latitude = {}, Longitude = {}".format(lat, lng))

except Exception as e:
        print("Error: ", str(e))

Enter City name that has to be searched : Ahmedabad


C:\Users\Nirav\AppData\Local\Temp/ipykernel_7436/2569291744.py:3: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search_bar = driver.find_element_by_id("searchboxinput")
C:\Users\Nirav\AppData\Local\Temp/ipykernel_7436/2569291744.py:7: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search_btn = driver.find_element_by_id("searchbox-searchbutton")


URL Extracted:  https://www.google.co.in/maps/place/Ahmedabad,+Gujarat/@23.0206289,72.2995341,10z/data=!3m1!4b1!4m5!3m4!1s0x395e848aba5bd449:0x4fcedd11614f6516!8m2!3d23.022505!4d72.5713621
Latitude = 23.0206289, Longitude = 72.2995341


In [83]:
driver.close()

# Write a program to scrap details of all the funding deals for second quarter (i.e Jan 21 – March 21)
from trak.in.

In [84]:
driver=webdriver.Chrome("chromedriver.exe")
time.sleep(2)

C:\Users\Nirav\AppData\Local\Temp/ipykernel_7436/1659935237.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome("chromedriver.exe")


In [85]:
driver.get("https://trak.in/")

In [86]:
#to get xpath for funding deals
fund_button = driver.find_element_by_xpath('//li[@id="menu-item-51510"]/a').get_attribute('href')
driver.get(fund_button)

C:\Users\Nirav\AppData\Local\Temp/ipykernel_7436/147889797.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  fund_button = driver.find_element_by_xpath('//li[@id="menu-item-51510"]/a').get_attribute('href')


In [87]:
# Empty lists
fund_dict = {}
fund_dict['Date'] = []
fund_dict['Startup Name'] = []
fund_dict['Industry/Vertical'] = []
fund_dict['Sub-Vertical'] = []
fund_dict['Location'] = []
fund_dict['Investor'] = []
fund_dict['Investment Type'] = []
fund_dict['Amount(in USD)'] = []


for i in range(54,57):
    # Date
    dt = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[2]'.format(i))
    for d in dt:
        fund_dict['Date'].append(d.text)

    # Startup Name
    sn = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[3]'.format(i))
    for n in sn:
        fund_dict['Startup Name'].append(n.text)
    
    # Industry/Vertical
    ind = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[4]'.format(i))
    for n in ind:
        fund_dict['Industry/Vertical'].append(n.text)
    
    # Sub-Vertical
    sv = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[5]'.format(i))
    for s in sv:
        fund_dict['Sub-Vertical'].append(s.text)

    # Location
    loc = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[6]'.format(i))
    for l in loc:
        fund_dict['Location'].append(l.text)
    
    # Investor
    inv = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[7]'.format(i))
    for n in inv:
        fund_dict['Investor'].append(n.text)
    
    # Investment Type
    invt = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[8]'.format(i))
    for n in invt:
        fund_dict['Investment Type'].append(n.text)
    
    # Amount
    amt = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[9]'.format(i))
    for a in amt:
        fund_dict['Amount(in USD)'].append(a.text)
    
fund_df = pd.DataFrame(fund_dict)
fund_df

C:\Users\Nirav\AppData\Local\Temp/ipykernel_7436/2789129144.py:15: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  dt = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[2]'.format(i))
C:\Users\Nirav\AppData\Local\Temp/ipykernel_7436/2789129144.py:20: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  sn = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[3]'.format(i))
C:\Users\Nirav\AppData\Local\Temp/ipykernel_7436/2789129144.py:25: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  ind = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[4]'.format(i))
C:\Users\Nirav\AppData\Local\Temp/ipykernel_7436/2789129144.py:30: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, va

,Date,Startup Name,Industry/Vertical,Sub-Vertical,Location,Investor,Investment Type,Amount(in USD)
0,15/01/2021,Digit Insurance,Financial Services,Insurance Services,Bengaluru,"A91 Partners, Faering Capital, TVS Capital Funds",Venture,"1,80,00,000"
1,28/01/2021,Bombay Shaving Company,Consumer Goods Company,"Shave care, beard care, and skincare products",New Delhi,Reckitt Benckiser,Venture,"6,172,258.50"
2,19/01/2021,DeHaat,AgriTech Startup,online marketplace for farm products and services,Patna,Prosus Ventures,Series C,"30,000,000"
3,19/01/2021,Darwinbox,SaaS,HR Tech,Mumbai,Salesforce Ventures,Seed,"15,000,000"
4,18/01/2021,mfine,Health Tech Startup,AI-powered telemedicine mobile app,Bengaluru,Heritas Capital Management,Venture Round,"16,000,000"
5,18/01/2021,Udayy,EdTech,Online learning platform for kids in class 1-5,Gurgaon,Sequoia Capital,Seed Funding,"15,000,000"
6,11/01/2021,True Elements,Food Startup,Whole Food plant based Nashta,Pune,SIDBI Venture Capital,Series,"100,000,000"
7,13/01/2021,Saveo,B2B E-commerce,Pharmacies,Bengaluru,"Matrix Partners India, RTP Global, others",Seed,"4,000,000"
8,11/02/2021,Doubtnut,Edu Tech,E-Learning Platform,Gurgaon,"SIG Global, Sequoia Capital, WaterBridge Ventu...",Series B,"2,500,000"
9,22/02/2021,Zomato,Hospitality,Online Food Delivery Platform,Gurgaon,"Tiger Global, Kora",Venture,"250,000,000"


In [89]:
driver.close()

# Write a program to scrap all the available details of best gaming laptops from digit.in.

In [90]:
driver=webdriver.Chrome("chromedriver.exe")
time.sleep(2)

C:\Users\Nirav\AppData\Local\Temp/ipykernel_7436/1659935237.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome("chromedriver.exe")


In [91]:
driver.get("https://www.digit.in/")
time.sleep(3)

In [92]:
#to search for best laptop
best_gam_lap = driver.find_element_by_xpath("//div[@class='listing_container']//ul//li[9]").click()
time.sleep(4)

C:\Users\Nirav\AppData\Local\Temp/ipykernel_7436/2071038886.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  best_gam_lap = driver.find_element_by_xpath("//div[@class='listing_container']//ul//li[9]").click()


In [93]:

#Creating empty lists
lap_name = []
ope_sys = []
display = []
processor = []
memory = []
weight = []
dimensions = []
graph_proc = []
price = []
# to Scrape laptop names
name_tags = driver.find_elements_by_xpath("//table[@id='summtable']//tr//td[1]")
for name in name_tags:
    lap_name.append(name.text)
    
    
# to Scrape the operating system
try:
    os_tags = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[3]//td[3]")
    for os in os_tags:
        ope_sys.append(os.text)
except NoSuchElementException:
    pass


# to Scrape display
try:
    disp_tags = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[4]//td[3]")
    for disp in disp_tags:
        display.append(disp.text)
except NoSuchElementException:
    pass


#to Scrape Processor
try:
    pro_tags = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[5]//td[3]")
    for pro in pro_tags:
        processor.append(pro.text)
except NoSuchElementException:
    pass


# to Scrape memory
try:
    memo_tags = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[6]//td[3]")
    for memo in memo_tags:
        memory.append(memo.text)
except NoSuchElementException:
    pass


# to Scrape weight
try:
    wgt_tags = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[7]//td[3]")
    for wgt in wgt_tags:
        weight.append(wgt.text)
except NoSuchElementException:
    pass


#to Scrape dimensions
try:
    dim_tags = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[8]//td[3]")
    for dim in dim_tags:
        dimensions.append(dim.text)
except NoSuchElementException:
    pass


# to Scrape Graph processor
try:
    gra_tags = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[9]//td[3]")
    for gra in gra_tags:
        graph_proc.append(gra.text)
except NoSuchElementException:
    pass


#to Scrape price
try:
    pri_tags = driver.find_elements_by_xpath("//td[@class='smprice']")
    for pri in pri_tags:
        price.append(pri.text.replace('₹','Rs '))
except NoSuchElementException:
    pass

C:\Users\Nirav\AppData\Local\Temp/ipykernel_7436/550403602.py:12: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  name_tags = driver.find_elements_by_xpath("//table[@id='summtable']//tr//td[1]")
C:\Users\Nirav\AppData\Local\Temp/ipykernel_7436/550403602.py:19: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  os_tags = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[3]//td[3]")
C:\Users\Nirav\AppData\Local\Temp/ipykernel_7436/550403602.py:28: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  disp_tags = driver.find_elements_by_xpath("//div[@class='Spcs-details']//tr[4]//td[3]")
C:\Users\Nirav\AppData\Local\Temp/ipykernel_7436/550403602.py:37: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  pro_t

In [94]:
# to make dataframe
Gaming_Laptop=pd.DataFrame({})
Gaming_Laptop['Laptop Name'] = lap_name
Gaming_Laptop['Operating system'] = ope_sys
Gaming_Laptop['Display'] = display
Gaming_Laptop['Processor'] = processor
Gaming_Laptop['Memory'] = memory
Gaming_Laptop['Weight'] = weight
Gaming_Laptop['Dimensions'] = dimensions
Gaming_Laptop['Graphical Processor'] = graph_proc
Gaming_Laptop['Price'] = price

In [95]:
#Printing data frame
Gaming_Laptop

,Laptop Name,Operating system,Display,Processor,Memory,Weight,Dimensions,Graphical Processor,Price
0,MSI Raider GE76,Windows 11 Home,"17"" (3840 x 2160)",12th Gen Intel Core i9-12900HK | 5 GHz,2 TB SSD/16 GBGB DDR5,2.9,397 x 284 x 26,NVIDIA GeForce RTX 3080Ti,"Rs 429,940"
1,ASUS ROG Strix Scar 15,Windows 11 Home,"15.6"" (2560 x 1440)",12th Gen Intel Core i9-12900H | 2.5 GHz,2 TB SSD/32 GBGB DDR5,2.3,259 x 354 x 27,NVIDIA GeForce RTX 3070 Ti,"Rs 285,390"
2,Acer Nitro 5,Windows 10,"15.6"" (1920 x 1080)",AMD Ryzen 9 Octa Core | 2.4 GHz,1 TB HDD/16 GBGB DDR4,2.4,363.4 x 255 x 23.9,NVIDIA GeForce RTX 3070,"Rs 129,990"
3,MSI Stealth 15M,Windows 10,"15.6"" (1920 x 1080)",Intel Core i7 11th Gen - 11375H | NA,1 TB SSD/16 GBGB DDR4,1.7,358.3 x 248 x 16.15,NVIDIA GeForce RTX 3060,"Rs 134,990"
4,ASUS ROG Strix Scar 15,Windows 10,"15.6"" (2560 x 1440)",AMD Ryzen 9 Octa Core - 5900HX | 3.3 GHz,2 TB SSD/32 GBGB DDR4,2.30,354 x 259 x 22.6,NVIDIA GeForce RTX 3080,"Rs 193,990"
5,ASUS ROG Strix Scar 15,Windows 10 Home,"15.6"" (1920 x 1080)",AMD Ryzen™ 9 5900HX | 3.3 GHz,1 TB SSD/16 GBGB DDR4,2.30,35.4 x 25.9 x 2.26,NVIDIA® GeForce RTX™ 3070,"Rs 215,990"
6,ASUS ZEPHYRUS G14,Windows 10 Home,"14"" (1920 x 1080)",AMD 3rd Gen Ryzen 9 | 3.3 GHz,1 TB SSD/16 GBGB DDR4,1.65,32.5 x 22.1 x 1.8,NVIDIA GeForce RTX 2060,"Rs 144,990"
7,HP Omen 16,Windows 11 Home,"16.1"" (1920 x 1080)",12th Gen Intel Core i7-12700H | 4.7 GHz,1 TB SSD/16 GBGB DDR4,2.32,36.92 x 24.8 x 2.3,NVIDIA GeForce RTX 3060,"Rs 139,990"
8,ASUS ROG ZEPHYRUS DUO 15,Windows 10,"15.6"" (3840 x 1100)",Intel Core i7 10th Gen 10875H | NA,512 GB SSD/4 GBGB DDR4,2.4,268.30 x 360.00 x 20.90,NVIDIA GeForce RTX 2070 Max-Q,"Rs 185,000"
9,Acer Aspire 7 gaming laptop,Windows 10 Home,"15.6"" (1920 x 1080)",AMD Ryzen™ 5-5500U hexa-core | NA,512 GB SSD/8 GBGB DDR4,2.15,2.29 x 36.3 x 25.4,NVIDIA® GeForce® GTX 1650,"Rs 53,490"


In [97]:
driver.close()

# Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be
scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

In [98]:
driver=webdriver.Chrome("chromedriver.exe")
time.sleep(2)

C:\Users\Nirav\AppData\Local\Temp/ipykernel_7436/1659935237.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome("chromedriver.exe")


In [99]:
driver.get("https://www.forbes.com/?sh=69e6b8c92254")

In [100]:
time.sleep(2)

In [101]:
#clicking the explore button
button = driver.find_element_by_xpath("//button[@class='icon--hamburger']")
button.click()
time.sleep(1)

C:\Users\Nirav\AppData\Local\Temp/ipykernel_7436/680114930.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  button = driver.find_element_by_xpath("//button[@class='icon--hamburger']")


In [102]:
#select billioners 
billioners = driver.find_element_by_xpath("/html/body/div[1]/header/nav/div[3]/ul/li[1]")
billioners.click()
time.sleep(1)

C:\Users\Nirav\AppData\Local\Temp/ipykernel_7436/1239588833.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  billioners = driver.find_element_by_xpath("/html/body/div[1]/header/nav/div[3]/ul/li[1]")


In [103]:
#select world billioners 
world_billioners= driver.find_element_by_xpath("/html/body/div[1]/header/nav/div[3]/ul/li[1]/div[2]/ul/li[2]/a")
world_billioners.click()
time.sleep(1)

C:\Users\Nirav\AppData\Local\Temp/ipykernel_7436/1195072266.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  world_billioners= driver.find_element_by_xpath("/html/body/div[1]/header/nav/div[3]/ul/li[1]/div[2]/ul/li[2]/a")


In [104]:
# empty lists
Rank = []
Person_Name = []
total_net_worth = []
Age = []
citizenship = []
Source = []
industry = []


while(True):
    #to Scrape rank
    rank_tags= driver.find_elements_by_xpath("//div[@class='rank']")
    for rank in rank_tags:
        Rank.append(rank.text)
    time.sleep(1)
    
    
    #to Scrape names
    name_tags= driver.find_elements_by_xpath("//div[@class='personName']//div")
    for name in name_tags:
        Person_Name.append(name.text)
    time.sleep(1)
    
    
    #to Scrape age
    age_tags= driver.find_elements_by_xpath("//div[@class='age']//div")
    for age in age_tags:
        Age.append(age.text)   
    time.sleep(1)
    
    
    #to Scrape citizenship
    cit_tags= driver.find_elements_by_xpath("//div[@class='countryOfCitizenship']")
    for cit in cit_tags:
        citizenship.append(cit.text)
    time.sleep(1)
    
    
    #to Scrape source of income
    sour_tags= driver.find_elements_by_xpath("//div[@class='source']")
    for sour in sour_tags:
        Source.append(sour.text)    
    time.sleep(1)
    
    
    #to Scrape Industry
    ind_tags= driver.find_elements_by_xpath("//div[@class='category']//div")
    for ind in ind_tags:
        industry.append(ind.text)
        
        
    #to scrape net_worth of billionaire 
    net_tags= driver.find_elements_by_xpath("//div[@class='netWorth']//div[1]")
    for net in net_tags:
        total_net_worth.append(net.text)
    time.sleep(1)
    
    
    #to Click on next button
    try:
        next_button = driver.find_element_by_xpath("//button[@class='pagination-btn pagination-btn--next ']")
        next_button.click()
    except:
        break
        
#to Scrape net worth
Net_Worth = []
for i in range(0,len(total_net_worth),2):
    Net_Worth.append(total_net_worth[i])

C:\Users\Nirav\AppData\Local\Temp/ipykernel_7436/3980716467.py:13: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  rank_tags= driver.find_elements_by_xpath("//div[@class='rank']")
C:\Users\Nirav\AppData\Local\Temp/ipykernel_7436/3980716467.py:20: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  name_tags= driver.find_elements_by_xpath("//div[@class='personName']//div")
C:\Users\Nirav\AppData\Local\Temp/ipykernel_7436/3980716467.py:27: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  age_tags= driver.find_elements_by_xpath("//div[@class='age']//div")
C:\Users\Nirav\AppData\Local\Temp/ipykernel_7436/3980716467.py:34: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  cit_tags= driver.find_elements_by_xpath("//div[

In [105]:
#Framing data
Billionaires=pd.DataFrame({})
Billionaires['Rank'] = Rank
Billionaires['Names'] = Person_Name
Billionaires['Net Worth'] = Net_Worth
Billionaires['Age'] = Age
Billionaires['Citizenship'] = citizenship
Billionaires['Source'] = Source
Billionaires['Industry'] = industry

In [106]:
#Printing dataframe
Billionaires

,Rank,Names,Net Worth,Age,Citizenship,Source,Industry
0,1.,Elon Musk,$219 B,50,United States,"Tesla, SpaceX",Automotive
1,2.,Jeff Bezos,$171 B,58,United States,Amazon,Technology
2,3.,Bernard Arnault & family,$158 B,73,France,LVMH,Fashion & Retail
3,4.,Bill Gates,$129 B,66,United States,Microsoft,Technology
4,5.,Warren Buffett,$118 B,91,United States,Berkshire Hathaway,Finance & Investments
...,...,...,...,...,...,...,...
195,192.,Marcel Herrmann Telles,$10.3 B,72,Brazil,beer,Food & Beverage
196,197.,Leon Black,$10 B,70,United States,private equity,Finance & Investments
197,197.,Joe Gebbia,$10 B,40,United States,Airbnb,Technology
198,197.,David Geffen,$10 B,79,United States,"movies, record labels",Media & Entertainment


In [140]:
driver.close()

# Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted
from any YouTube Video

In [148]:
driver=webdriver.Chrome("chromedriver.exe")
time.sleep(2)

C:\Users\Nirav\AppData\Local\Temp/ipykernel_7436/1659935237.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome("chromedriver.exe")


In [149]:
driver.get("https://www.youtube.com/")
time.sleep(2)

In [150]:
# input the keywords that user wants to search
user_input = input('Enter the video you want to search : ')

#finding element for search bar
search_bar = driver.find_element_by_xpath('//input[@id="search"]')
search_bar.send_keys(user_input)
time.sleep(1)

Enter the video you want to search : bahubali


C:\Users\Nirav\AppData\Local\Temp/ipykernel_7436/2837691139.py:5: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  search_bar = driver.find_element_by_xpath('//input[@id="search"]')


In [151]:
#clicking on search button
search_btn = driver.find_element_by_id("search-icon-legacy")  
search_btn.click()
time.sleep(1)

C:\Users\Nirav\AppData\Local\Temp/ipykernel_7436/874414561.py:2: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search_btn = driver.find_element_by_id("search-icon-legacy")


In [152]:
#clicking on first video
link_click = driver.find_element_by_xpath("//yt-formatted-string[@class ='style-scope ytd-video-renderer']")
link_click.click()

# 10000 time we scroll down to generate more Comments
for _ in range(10000):
    driver.execute_script("window.scrollBy(0,10000)")

C:\Users\Nirav\AppData\Local\Temp/ipykernel_7436/2087031875.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  link_click = driver.find_element_by_xpath("//yt-formatted-string[@class ='style-scope ytd-video-renderer']")


In [153]:
# empty lists
comments = []
comment_time = []
Time = []
Likes = []
No_of_Likes = []

# to scrape comments
cm_tags = driver.find_elements_by_id("content-text")
for cm in cm_tags:
    if cm.text is None:
        comments.append("--")
    else:
        comments.append(cm.text)
time.sleep(5)
comments[:500]

C:\Users\Nirav\AppData\Local\Temp/ipykernel_7436/862396452.py:9: DeprecationWarning: find_elements_by_id is deprecated. Please use find_elements(by=By.ID, value=id_) instead
  cm_tags = driver.find_elements_by_id("content-text")


['Still this movie gives vibes ',
 '"I respect everyone who were involved in this, Seriously the best piece that I ever seen on Youtube, Hats OFF To YOU I LOVE YOUR VIDEO S!!!!"" ',
 "Nothing comes close to Bahubali. It's at another level ️",
 'A milestone in Indian film making.\nCenter of gravity shifted South after this movie.',
 'Rajamouli sir+prashanth sir = Bollywood RIP ',
 'Why people is comparing bahubali with other films. Bahubali was made in 2015.In that time It was the best film in vfx, story, character, thrill and other stuffs.It is an emotion now.No other film can break it.',
 "Its 2022 and still can't get over from this movie ",
 'The best film\nThe best content\nThe best journey\nThe best visual wonder',
 'Historic movie no one can create such a masterpiece movie ever.. Hats off to south director, writer, artists .. lost of love and huge fan from Nepal',
 'Thanks bruh.. I was searching for this movie for like 3 months.. Finally I found this masterpiece ',
 'If Bahubali s

In [154]:
len(comments)

649

In [155]:
# to scrape time when comment was posted
tm_tags = driver.find_elements_by_xpath("//a[contains(text(),'ago')]")
for tm in tm_tags:
    if tm.text is None:
        Time.append("--")
    else:
        Time.append(tm.text)

for i in range(0,len(Time),2):
    comment_time.append(Time[i])
time.sleep(5)
comment_time[:500]

# to scrape comment likes
like_tags = driver.find_elements_by_xpath("//span[@class='style-scope ytd-comment-action-buttons-renderer']")
for like in like_tags:
    if like.text is None:
        Likes.append("--")
    else:
        Likes.append(like.text)
    
for i in range(1,len(Likes),2):
    No_of_Likes.append(Likes[i])
Likes[:500]    

C:\Users\Nirav\AppData\Local\Temp/ipykernel_7436/3880725983.py:2: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  tm_tags = driver.find_elements_by_xpath("//a[contains(text(),'ago')]")
C:\Users\Nirav\AppData\Local\Temp/ipykernel_7436/3880725983.py:15: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  like_tags = driver.find_elements_by_xpath("//span[@class='style-scope ytd-comment-action-buttons-renderer']")


['',
 '37',
 '',
 '1',
 '',
 '121',
 '',
 '87',
 '',
 '144',
 '',
 '137',
 '',
 '125',
 '',
 '117',
 '',
 '60',
 '',
 '50',
 '',
 '27',
 '',
 '10',
 '',
 '212',
 '',
 '3',
 '',
 '151',
 '',
 '7',
 '',
 '168',
 '',
 '42',
 '',
 '36',
 '',
 '2',
 '',
 '51',
 '',
 '1',
 '',
 '108',
 '',
 '14',
 '',
 '6',
 '',
 '3',
 '',
 '112',
 '',
 '60',
 '',
 '39',
 '',
 '1',
 '',
 '52',
 '',
 '1',
 '',
 '9',
 '',
 '7',
 '',
 '',
 '',
 '27',
 '',
 '3',
 '',
 '3',
 '',
 '6',
 '',
 '2',
 '',
 '12',
 '',
 '51',
 '',
 '39',
 '',
 '28',
 '',
 '2',
 '',
 '29',
 '',
 '16',
 '',
 '3',
 '',
 '6',
 '',
 '',
 '',
 '22',
 '',
 '42',
 '',
 '3',
 '',
 '11',
 '',
 '',
 '',
 '1',
 '',
 '52',
 '',
 '2',
 '',
 '18',
 '',
 '2',
 '',
 '3',
 '',
 '45',
 '',
 '17',
 '',
 '',
 '',
 '4',
 '',
 '37',
 '',
 '3',
 '',
 '18',
 '',
 '11',
 '',
 '8',
 '',
 '3',
 '',
 '2',
 '',
 '37',
 '',
 '29',
 '',
 '27',
 '',
 '1',
 '',
 '2',
 '',
 '1',
 '',
 '12',
 '',
 '14',
 '',
 '86',
 '',
 '1',
 '',
 '2',
 '',
 '2',
 '',
 '',
 '',
 '',
 '',

In [159]:
#Creating dataframe
Youtube=pd.DataFrame({})
Youtube['Comments'] = comments[:325]
Youtube['Comment_time'] = comment_time[:325]
Youtube['Comment upvotes'] = No_of_Likes[:325]
#taking 325 comments in data frame because we have 325 likes
#Printing dataframe
Youtube

,Comments,Comment_time,Comment upvotes
0,Still this movie gives vibes,2 weeks ago,37
1,"""I respect everyone who were involved in this,...",3 weeks ago,1
2,Nothing comes close to Bahubali. It's at anoth...,1 month ago,121
3,A milestone in Indian film making.\nCenter of ...,1 month ago,87
4,Rajamouli sir+prashanth sir = Bollywood RIP,1 month ago,144
...,...,...,...
320,Wow,1 month ago,4
321,️️,1 month ago,1
322,Bahhubali what a movie yar,1 month ago,
323,"Dude, how come after 2 weeks though, copyright...",3 weeks ago,4


In [200]:
driver.close()

# Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in
“London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews,
overall reviews, privates from price, dorms from price, facilities and property description.

In [236]:
driver=webdriver.Chrome("chromedriver.exe")
time.sleep(2)

C:\Users\Nirav\AppData\Local\Temp/ipykernel_7436/1659935237.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome("chromedriver.exe")


In [237]:
driver.get("https://www.hostelworld.com/")

In [240]:
#to scrape location search bar
search_loc = driver.find_element_by_class_name('search-input')
search_loc.send_keys("London")
time.sleep(2)

C:\Users\Nirav\AppData\Local\Temp/ipykernel_7436/214785678.py:2: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  search_loc = driver.find_element_by_class_name('search-input')


In [241]:
#select london
london = driver.find_element_by_xpath('/html/body/div[3]/div/div/div[1]/div[1]/div/div[2]/div[4]/div/div[2]/div/div[1]/div/div/ul/li[2]')
london.click()
time.sleep(2)

C:\Users\Nirav\AppData\Local\Temp/ipykernel_7436/661299404.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  london = driver.find_element_by_xpath('/html/body/div[3]/div/div/div[1]/div[1]/div/div[2]/div[4]/div/div[2]/div/div[1]/div/div/ul/li[2]')


In [242]:
# do click in search button
search_btn = driver.find_element_by_id("search-button")
search_btn.click()

C:\Users\Nirav\AppData\Local\Temp/ipykernel_7436/3057431681.py:2: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search_btn = driver.find_element_by_id("search-button")


In [243]:
title = []
distance = []
rating = []
total_review = []
prices = []
private_price = []
dorms_price = []
facilities=[]
prop_details=[]

title_tags = driver.find_elements_by_xpath("//h2[@class='title title-6']")
for i in title_tags:
    title.append(i.text)
    
distance_tags = driver.find_elements_by_xpath("//span[@class='description']")
for j in distance_tags:
    distance.append(j.text.split('-')[1])

rating_tags = driver.find_elements_by_xpath("//div[@class='score orange big']")
for k in rating_tags:
    rating.append(k.text)

total_review_tags = driver.find_elements_by_xpath("//div[@class='reviews']")
for l in total_review_tags:
    total_review.append(l.text)

    
P_prices_tags=driver.find_elements_by_xpath("//div[1][@class='price-col']") #locating web element of price
    
for n in P_prices_tags:
    private_price.append(n.text)


D_prices_tags=driver.find_elements_by_xpath("//div[2][@class='price-col']") #locating web element of price
  
for o in D_prices_tags:
    dorms_price.append(o.text)

faci_tags=driver.find_elements_by_xpath("//div[@class='facilities-label facilities']")    
for p in faci_tags:
    facilities.append(p.text)

prop_des=driver.find_elements_by_xpath("//div[@class='title-row']")    
for q in prop_des:
    prop_details.append(q.text)    
    

C:\Users\Nirav\AppData\Local\Temp/ipykernel_7436/577004488.py:11: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  title_tags = driver.find_elements_by_xpath("//h2[@class='title title-6']")
C:\Users\Nirav\AppData\Local\Temp/ipykernel_7436/577004488.py:15: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  distance_tags = driver.find_elements_by_xpath("//span[@class='description']")
C:\Users\Nirav\AppData\Local\Temp/ipykernel_7436/577004488.py:19: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  rating_tags = driver.find_elements_by_xpath("//div[@class='score orange big']")
C:\Users\Nirav\AppData\Local\Temp/ipykernel_7436/577004488.py:23: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  total_review_tags = driver.

In [209]:
for i in driver.find_elements_by_xpath("//div[@class = 'pagination-item pagination-current' or @class='pagination-item']"):
    i.click()
    time.sleep(4)
    #to get hostel name
    try:
        name = driver.find_elements_by_xpath("//h2[@class='title title-6']")
        for i in name:
            hostel_name.append(i.text)
    except NoSuchElementException:
        hostel_name.append('-')
    #to get distance from city centre
    
    try:
        dist = driver.find_elements_by_xpath("//div[@class='subtitle body-3']//a//span[1]")
        for i in dist:
            distance.append(i.text.replace('Hostel - ',''))
    except NoSuchElementException:
        distance.append('-')
        
    for i in driver.find_elements_by_xpath("//div[@class='prices-col']"):
    #to get privates from price
        try:
            pvt_price = driver.find_element_by_xpath("//a[@class='prices']//div[1]//div")
            pvt_prices.append(pvt_price.text)
        except NoSuchElementException:
            pvt_prices.append('-')
    #to get  dorms from price
    for i in driver.find_elements_by_xpath("//div[@class='prices-col']"):
        try:
            dorms = driver.find_element_by_xpath("//a[@class='prices']//div[2]//div")
            dorms_price.append(dorms.text)
        except NoSuchElementException:
            dorms_price.append('-')
            #to get facilities
    try:
        fac1 = driver.find_elements_by_xpath("//div[@class='has-wifi']")
        fac2 = driver.find_elements_by_xpath("//div[@class='has-sanitation']")
        for i in fac1:
            for j in fac2:
                facilities.append(i.text +', '+ j.text )
    except NoSuchElementException:
        facilities.append('-')
    #to fetch url of each hostel
    p_url = driver.find_elements_by_xpath("//div[@class='prices-col']//a[2]")
    for i in p_url:
        product_url.append(i.get_attribute('href'))

for i in product_url:
    driver.get(i)
    time.sleep(3)
    #to click on show more button for description
    try:
        driver.find_element_by_xpath("//a[@class='toggle-content']").click()
        time.sleep(5)
    except NoSuchElementException:
        pass
    #to get ratings
    try:
        rat = driver.find_element_by_xpath("//div[@class='score orange big' or @class='score gray big']")
        rating.append(rat.text)
    except NoSuchElementException:
        rating.append('-')
    # to get total reviews
        
    try:
        rws = driver.find_element_by_xpath("//div[@class='reviews']")
        reviews.append(rws.text.replace('Total Reviews',''))
    except NoSuchElementException:
        reviews.append('-')
        #to get  overall review
    try:
        overall_rw = driver.find_element_by_xpath("//div[@class='keyword']//span")
        over_all.append(overall_rw.text)
    except NoSuchElementException:
        over_all.append('-')
    #to get property description 
    try:
        disc = driver.find_element_by_xpath("//div[@class='content']")
        description.append(disc.text)
    except NoSuchElementException:
        over_all.append('-')


C:\Users\Nirav\AppData\Local\Temp/ipykernel_7436/369944220.py:1: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  for i in driver.find_elements_by_xpath("//div[@class = 'pagination-item pagination-current' or @class='pagination-item']"):


In [244]:
df = pd.DataFrame({'Hostel Name':title[:5],'Distance':distance[:5],'Rating':rating[:5],'Total Review':total_review[:5],'Private Price':private_price[:5], 'Dorms Price':dorms_price[:5],'Property Details':prop_details[:5],'Facilities':facilities[:5]})
df 

,Hostel Name,Distance,Rating,Total Review,Private Price,Dorms Price,Property Details,Facilities
0,St Christopher's Village,1.8km from city centre,7.9,11206 Total Reviews,No Privates Available,Dorms From\nRs4089.39 Rs3680,St Christopher's Village\nHostel - 1.8km from ...,Free WiFi\nFollows Covid-19 sanitation guidance
1,Selina Camden,5.5km from city centre,9.6,24 Total Reviews,Privates From\nRs25195.19 Rs21416,No Dorms Available,Selina Camden\nHostel - 5.5km from city centre,Free WiFi
2,Saint James Backpackers,5.5km from city centre,6.0,1789 Total Reviews,Privates From\nRs17191.06 Rs16332,Dorms From\nRs8229.91 Rs7818,St Christopher's Village\nHostel - 1.8km from ...,Free WiFi\nFree Breakfast\nFollows Covid-19 sa...
3,Barry House,4.2km from city centre,8.6,10 Total Reviews,Privates From\nRs22357,No Dorms Available,Selina Camden\nHostel - 5.5km from city centre...,Free WiFi\nFree Breakfast
4,Colliers Hotel,2.1km from city centre,7.4,4 Total Reviews,Privates From\nRs12960,No Dorms Available,Saint James Backpackers\nHostel - 5.5km from c...,Follows Covid-19 sanitation guidance


In [246]:
driver.close()